In [1]:
# conda activate Ollama_RPG
# systemctl restart ollama


In [ ]:
%pip install requests

##### Bibliotecas e funções basicas

In [1]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel
from typing import Optional
from typing import List
import requests
import random
import json # se precisar manipular json
import html # para corrigir algumas saidas que vem com códigos de escape HTML, algo como &quot que representa " (aspas duplas).
import os
from chaves import groq_api_teste

In [2]:
os.environ["GROQ_API_KEY"] = groq_api_teste

In [3]:
# Model response Groq
def model_response(system_prompt: str, user_prompt: str, schema_json: str, output_class: any, temperature: float = 1.0, max_attempts: int = 1, model: str = "llama-3.1-8b-instant") -> Optional[any]:
    """
    Send requests to Groq API and parse the response into a Pydantic model.
    
    Args:
        system_prompt (str): System instructions
        user_prompt (str): User input
        schema_json (str): JSON schema for output validation
        output_class (any): Pydantic model class for output
        temperature (float): Temperature parameter for response generation
        max_attempts (int): Maximum number of retry attempts
        model (str): Groq model name to use
    """
    api_key = os.getenv("GROQ_API_KEY")
    if not api_key:
        raise ValueError("GROQ_API_KEY environment variable is not set")

    parser = PydanticOutputParser(pydantic_object=output_class)
    format_instructions = parser.get_format_instructions()
    
    messages = [
        {
            "role": "system",
            "content": f"{system_prompt}\n\nPlease provide responses in the following format:\n{format_instructions}"
        },
        {
            "role": "user",
            "content": f"{user_prompt}\n\n{schema_json}"
        }
    ]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "messages": messages,
        "model": model,
        "temperature": temperature,
        "max_tokens": 1024,
        "top_p": 1,
        "stream": False,
        "response_format": {
            "type": "json_object"
        }
    }

    attempts = 0
    while attempts < max_attempts:
        try:
            response = requests.post(
                "https://api.groq.com/openai/v1/chat/completions",
                headers=headers,
                json=payload
            )
            response.raise_for_status()

            response_data = response.json()
            response_content = response_data['choices'][0]['message']['content']
            return parser.parse(response_content)

        except Exception as e:
            attempts += 1
            if attempts == max_attempts:
                print(f"Failed after {max_attempts} attempts. Last error: {e}")
                return None
            print(f"Attempt {attempts} failed. Retrying...")

    return None

In [4]:
# Tabela de Tramas
Evento_inicial = [
    "Um exército de mortos avança",
    "Um tirano dominou terras",
    "Um mago controlou um grande mal",
    "Uma criatura poderosa despertou",
    "Um artefato sagrado foi roubado",
    "Um Rei bondoso foi assassinado",
    "Criaturas malignas estão se unindo",
    "Um local sagrado foi conspurcado",
    "Uma grande guerra se iniciou",
    "Um poderoso dragão surgiu",
    "Pessoas estão sendo assassinadas",
    "Um demônio invadiu nosso plano",
    "Está chovendo fogo dos céus",
    "Uma estranha torre surgiu do nada",
    "As divindades desapareceram",
    "As plantações estão morrendo",
    "Uma grande disputa iniciou",
    "Uma peste mágica se espalha",
    "Um velho louco contou algo",
    "Criaturas das trevas surgem a noite"
]

Acao_necessaria = [
    "Encontrar um item poderoso",
    "Despertar uma divindade",
    "Derrotar um Vilão",
    "Realizar uma grande viagem",
    "Invadir a poderosa fortaleza",
    "Resistir bravamente ao perigo",
    "Destruir algo perigoso",
    "Restabelecer a ordem",
    "Reunir um grande exército",
    "Resgatar o herói perdido",
    "Viajar para outro plano",
    "Desafiar a morte",
    "Caçar um monstro",
    "Encontrar algo perdido",
    "Descobrir um grande mistério",
    "Resistir ao medo e a loucura",
    "Buscar um componente raro",
    "Se aproximar furtivamente",
    "Estar sempre a frente",
    "Chegar em um local remoto"
]

Consequencia_terrivel = [
    "O mundo será destruído",
    "A escuridão tomará o mundo",
    "Um deus maligno despertará",
    "Suas terras serão tomadas",
    "Uma guerra eterna acontecerá",
    "Um reino de ódio começará",
    "Uma doença se espalhará",
    "Milhões morrerão",
    "O caos tomará o mundo",
    "A realidade irá colapsar",
    "Os deuses irão desaparecer",
    "Bestas dominarão o mundo",
    "Você perderá tudo o que tem",
    "A fome tomará o mundo",
    "Demônios tomarão o mundo",
    "Portais planares irão surgir",
    "Os mortos levantarão",
    "A magia desaparecerá",
    "Um mal passado retornará",
    "Uma noite eterna começará"
]

ambientacoes_possiveis = [
    "(Épica) Em um reino onde a magia floresce e ruínas antigas sussurram segredos de eras passadas, você trilha seu caminho como um aventureiro, desbravando masmorras sombrias e enfrentando feras lendárias em busca de glória e tesouros esquecidos.",
    "(Misteriosa) Sob um céu perpetuamente crepuscular, onde a linha entre o real e o fantástico se esvai, você se aventura por um mundo repleto de perigos ocultos e maravilhas etéreas, explorando masmorras amaldiçoadas e confrontando criaturas grotescas que espreitam nas sombras.",
    "(Ação) O mundo clama por heróis! Empunhando sua arma e sua coragem, você se lança em uma jornada através de terras assoladas por monstros e armadilhas mortais, invadindo masmorras repletas de tesouros e provando seu valor contra inimigos formidáveis." ]

In [5]:
# Tabela de Cenas
Lugar = [
    "uma vila pequena",
    "um reino grande",
    "uma masmorra mortal",
    "um outro plano de existência",
    "uma passagem perigosa",
    "uma fortaleza inexpugnável",
    "um local amaldiçoado",
    "um local ermo",
    "um local sagrado",
    "uma taverna movimentada",
    "uma viela escura",
    "uma paisagem magnífica",
    "um acampamento",
    "uma feira movimentada",
    "uma estrada durante uma viagem",
    "uma caverna inóspita",
    "um veículo em movimento",
    "uma tribo selvagem",
    "um labirinto",
    "um local desolado"
]

Personagem_grupo_rival = [
    "um aventureiro",
    "um grupo rival",
    "uma pessoa traiçoeira",
    "um velho aventureiro",
    "um mensageiro",
    "um ser muito poderoso",
    "um exército",
    "uma pessoa amaldiçoado",
    "um contratante misterioso",
    "um artefato inteligente",
    "uma pessoa muito rica",
    "um animal",
    "um conhecido do passado",
    "uma tropa se deslocando",
    "uma pessoa amiga",
    "uma pessoa familiar",
    "um batedor",
    "um espião",
    "um fantasma ou desmorto",
    "um viajante de terras distantes",
    "um jovem escudeiro",
    "um ajudante",
    "um familiar de um mago",
    "uma criatura inteligente"
]

Evento = [
    "O Vilão surge",
    "Bandidos buscam riquezas",
    "Um grande mistério surge",
    "Alguém foi morto aqui",
    "Uma armadilha a frente",
    "Um ataque surpresa",
    "Os servos do Vilão estão aqui",
    "Uma passagem difícil",
    "Uma passagem secreta",
    "Monstros errantes aparecem",
    "O Vilão ataca",
    "Você encontra pistas",
    "Você é cercado",
    "Uma armadilha a frente",
    "O Lugar está em perigo",
    "Monstros errantes aparecem",
    "O tempo muda drasticamente",
    "Os servos do Vilão estão aqui",
    "Algo o perturba",
    "Uma catástrofe acontece",
    "Você encontra um tesouro"
]

In [6]:
# Tabela de Raças
Racas = [# 12 Raças no total
    "Tiefling: Possui sangue infernal em suas veias (resistência ao fogo e visão no escuro)",
    "Gnomo: Perspicaz e astuto (resistência a ilusões e domínio de magias menores)",
    "Anão: Robusto e artesão nato (resistência a veneno e proficiência com ferramentas)",
    "Humano: Adaptável a toda situação e ambiente (receba Vantagem uma vez por Cena)",
    "Meio-Orc: Força e resistência sobre-humana (resistência a fadiga e ferimentos)",
    "Goliath: Forte como uma montanha (resistência ao frio e proficiência em alpinismo)",
    "Elfo: Graça e longevidade élfica (resistência a encantamentos e sentidos aguçados)",
    "Meio-Elfo: Versatilidade de dois mundos (escolha dois atributos para receber +1)",
    "Draconato: Herança dracônica (resistência elemental e proficiência com uma arma)",
    "Genasi: Afinidade elemental (resistência e proficiência relacionada ao elemento escolhido)",
    "Halfling: Pequeno e sortudo (furtividade e resistência a paralisia)",
    "Aasimar: Abençoado pelos celestiais (resistência radiante/necrótica e aura de cura)"
]

In [7]:
# Tabela de Classes
Classes = [ # 14 Classes
   "Ladino: Especialista em subterfúgio que vive em busca de riquezas, usando sua furtividade e astúcia",
   "Guerreiro: Mestre em combate versado em diversas armas e técnicas marciais",
   "Mago: Estudioso do oculto que controla poderosas magias através do conhecimento arcano",
   "Clérigo: Devoto divino que canaliza poder sagrado para espalhar a palavra de sua fé",
   "Monge: Lutador disciplinado que busca a perfeição através do equilíbrio entre corpo e mente",
   "Druida: Guardião da natureza que canaliza poderes selvagens para proteger as terras virgens",
   "Bárbaro: Feroz combatente que sobrevive nas terras selvagens através de sua força bruta",
   "Paladino: Soldado sagrado que luta incansavelmente por seus ideais e juramentos",
   "Artífice: Engenhoso inventor que combina magia e tecnologia para criar artefatos únicos",
   "Bruxo: Místico que obteve poderes através de um pacto com uma entidade extraplanar",
   "Patrulheiro: Batedor astuto especializado em rastrear e emboscar seus inimigos",
   "Bardo: Artista versátil que inspira aliados e eterniza as grandes histórias em canções",
   "Feiticeiro: Conjurador nato que manifesta magias poderosas através de seu próprio sangue",
   "Caçador de Sangue: Guerreiro sombrio que domina a arte proibida da magia sanguínea"
]

In [8]:
# Banco de ideias
Assunto = [
    "Um poderoso mago recluso",
    "Alguém importante está sumido",
    "Um herói do passado",
    "Um velho louco procura você",
    "Um distante templo sagrado",
    "Um exército se reúne",
    "Um povo de outrora",
    "Os bardos contam histórias",
    "Sinais de uma ruína perdida",
    "Uma guerra está acontecendo",
    "O caminho para um tesouro",
    "Uma doença se espalha",
    "Uma festa acontece",
    "Uma festa irá acontecer",
    "Uma fera deve ser caçada",
    "Uma mal cresce a cada dia",
    "Uma maldição foi lançada",
    "Os astros estão alinhados",
    "Uma grande mentira é contada",
    "Um tesouro está perdido",
    "Vultos estão cercando o local"
]

Acao = [
    "Viajar",
    "Unir/Reunir",
    "Prender",
    "Proteger",
    "Vencer/Derrotar",
    "Roubar/Resgatar",
    "Fugir",
    "Matar/Destruir",
    "Fortalecer",
    "Procurar",
    "Esconder",
    "Amar/Adorar",
    "Comer/Beber",
    "Construir",
    "Escalar/Subir",
    "Descansar/Parar",
    "Saltar/Descer",
    "Odiar/Temer",
    "Anular",
    "Teleportar"
]

Coisa_objeto = [
    "Tesouro",
    "Passagem secreta",
    "Cristal",
    "Sinal de violência",
    "Item mágico",
    "Rastros",
    "Portal",
    "Runa",
    "Ídolo/Estátua",
    "Animal mítico",
    "Roupas",
    "Gosma/Limo",
    "Fumaça/Névoa",
    "Garras/Escamas",
    "Garrafa/Poção",
    "Comida/Bebida",
    "Corpo/Destroço",
    "Dejetos",
    "Ferramentas",
    "Artefato"
]

Qualidade = [
    "Profano",
    "Frágil",
    "Bestial",
    "Real",
    "Divino",
    "Popular",
    "Perdido",
    "Mágico",
    "Resistente",
    "Sagrado",
    "Natural",
    "Pesado/Cheio",
    "Frio",
    "Invisível/Imaterial",
    "Nobre/Caro",
    "Energético",
    "Extraplanar",
    "Quente",
    "Leve/Vazio",
    "Ancestral"
]

In [9]:
# Para as cenas: Tem 4 lugares e e (eventos/personagens)
def cenas_aleatorias():
    """
    Seleciona as 4 cenas aleatórias para o RPG estruturadas em um dicionário por ato.
    Cada ato contém local, evento (opcional) e personagem (opcional).
    Retorna um dicionário onde as chaves são os atos e os valores são dicionários
    contendo 'lugar', 'evento' e 'personagem'.
    """
    atos = ["Introduction", "Development", "Preparatory Climax", "Climax"]
    
    # Sorteando os lugares
    lugares_sorteados = random.sample(range(len(Lugar)), 4)
    
    # Inicializando o dicionário de cenas
    cenas = {}
    
    # Preenchendo o dicionário para cada ato
    for i, ato in enumerate(atos):
        # Inicializando com valores padrão
        cenas[ato] = {
            'lugar': Lugar[lugares_sorteados[i]],
            'evento': None,
            'personagem': None
        }
        
        # Sorteando se terá evento ou personagem
        dado = random.randint(1, 20)
        if dado <= 10:
            # Sorteia um personagem
            P = random.randrange(len(Personagem_grupo_rival))
            cenas[ato]['personagem'] = Personagem_grupo_rival[P]
        else:
            # Sorteia um evento
            E = random.randrange(len(Evento))
            cenas[ato]['evento'] = Evento[E]
    
    return cenas

In [10]:
# Montar a Trama: seleciona a que aconteceu, o que precisa ser feito e as consequencias.


def cenario_aleatorio():
    AA = random.sample(range(len(Evento_inicial)), 1)[0]
    VP = random.sample(range(len(Acao_necessaria)), 1)[0]
    SN = random.sample(range(len(Consequencia_terrivel)), 1)[0]
    AM = random.sample(range(len(ambientacoes_possiveis)), 1)[0]
    return f"""Descrisão do Cenário:
- Tema principal: Fantasy
- Problema que aconteceu: {Evento_inicial[AA]}.
- O que precisa ser feito: {Acao_necessaria[VP]}. 
- Consequências: {Consequencia_terrivel[SN]}.
- Ambientação: {ambientacoes_possiveis[AM]}"""

In [11]:
# Função que divide um texto em linhas menores, sem cortar palavras no meio.
import textwrap

def dividir_texto_em_linhas(texto, largura=60):
    """
    Divide um texto longo em linhas menores, sem cortar palavras no meio.

    Parâmetros:
        texto (str): O texto a ser dividido.
        largura (int): A largura máxima de cada linha. Padrão é 60 caracteres.

    Retorna:
        str: O texto formatado com quebras de linha.
    """
    # Divide o texto em linhas respeitando o limite de largura
    linhas = textwrap.wrap(texto, width=largura)
    
    # Junta as linhas com quebras de linha
    return "\n".join(linhas)

Criação da Introdução do cenário

Detalhes sobre os persenagens.

In [12]:
system_prompt = """Você é um especialista em criação de histórias para RPG."""

promtpt_introducao = f"""Crie a introdução de uma aventura de RPG em um mundo de fantasia que ajude na construção das cenas. Com base na Descrição a seguir:

## Descrição:
{cenario}

## Instruções:
1. Descrição da ambientação do mundo, baseado na Descrição do cenário **Ambientação** e **Gênero**.
2. Descrição de como o mundo está agora, baseado na Descrição do cenário **Algo aconteceu**.
3. Descrição da ação necessária para resolver o estado atual, baseado na Descrição do cenário **Você precisa**.
4. Descrição das consequências caso o objetivo não seja cumprido, baseado na Descrição do cenário **Consequências**.
5. Não defina personagens nem eventos.
6. Defina um nome para os locais de acordo com a sua descrição, para os seguintes as seguintes descrisções: {locais}
7. Conecte as partes da historia de maneira logica e coerente.

## Saida
Você deve responder em formato JSON seguindo o seguinte esquema:\n"""


schema_json_string = "{\n"
schema_json_string += "  \"messages\": [\n"
schema_json_string += "    {\"number\": 1, \"content\": \"Descrição breve da ambientação do mundo (nome dos lugares, clima, cultura, povos) sem mensionar o que ocorrerá no futuro ou quem fará a mudança. Sempre iniciando com: 'Em um mundo onde ...'.\"},\n"
schema_json_string += "    {\"number\": 2, \"content\": \"Descrição breve como o mundo está **agora**, destacando o que mudou, e explicar em detalhes o evento ou personagem que levou o mundo a esse estado. Sempre iniciando com: 'Mas agora, ...'\"},\n"
schema_json_string += "    {\"number\": 3, \"content\": \"Descrição breve da ação necessária para resolver o estado atual, com no máximo 1 ação ou objetivo. Não inclua consequências do fracasso nem quem deve executar a ação. Sempre comece com: 'É necessário ...'\"},\n"
schema_json_string += "    {\"number\": 4, \"content\": \"Descrição breve das consequências caso o objetivo não seja cumprido, ligando ela **ao que aconteceu**.\"}],\n"
schema_json_string += "  \"antagonist\": \"Descrição do objetivo do antagonista, começando com: 'O objetivo do antagonista é...'\",\n"
schema_json_string += "  \"locations\": [\n"

for i in range(len(cenas[0])): 
    schema_json_string += f"    {{\"description\": \"{cenas[0][i]}\", \"name\": \"Nome para um local descrito como: '{cenas[0][i]}'\"}}"
    if i < len(cenas[0]) - 1: 
        schema_json_string += ","
    schema_json_string += "\n"

schema_json_string += "  ],\n"
schema_json_string += "  \"objective\": \"Informe brevemente o objetivo principal, começando com: 'O objetivo principal é ...'\"\n"
schema_json_string += "}"


introducao_schema = json.loads(schema_json_string)

class Message(BaseModel):
    number: int
    content: str

class Location(BaseModel):
    description: str
    name: str
    
# Classe que representa o cenário completo
class Story(BaseModel):
    messages: List[Message]
    antagonist: str
    locations: List[Location]
    objective: str

story = model_response(system_prompt, promtpt_introducao, introducao_schema, Story)

print(cenario)
print("\nIntrodução:")
for message in story.messages:
    print(dividir_texto_em_linhas(message.content, 100))
    print()
print(f"Objetivo principal: {story.objective}")
print(f"\nAntagonista: {story.antagonist}")
print(f"\nLocais:")
for location in story.locations:
    print(f">> {location.name}: {location.description}")

NameError: name 'cenario' is not defined

In [ ]:
system_prompt = "Você é um especialista em criação de histórias para RPG."

personagens_descrisao = ""
for i in range(len(cenas[1])):
   personagens_descrisao += f"{cenas[1][i][1]}; "
personagens_descrisao += f"O Antagonista, {story.antagonist}."
print(personagens_descrisao)

promtpt = f"""Para todos os personagens, escolha um nome, papel, motivation, personality e Connection to Tram de acordo com a sua importância na história.

## Introdução:
1. **Ambientação**:   
{story.messages[0].content}

2. **O que aconteceu**: 
{story.messages[1].content}

3. **Como resolver**: 
{story.messages[2].content}

4. **Consequências de Falhar**:  
{story.messages[3].content}

5. **Objetivo Principal**:
{story.objective}

6. **Tabela de Elementos**:

| **Prefixo**        | **Raiz**         | **Sufixo**      |
|---------------------|------------------|-----------------|
| Aure (Dourado)     | Magnus (Grande)  | ius (Homem)     |
| Luci (Luz)         | Noct (Noite)     | us (Ser)        |
| Cor (Coração)      | Fulg (Brilho)    | arius (Guardião)|
| Flavi (Amarelo)    | Tempest (Tempestade) | or (Aquele que) |
| Cael (Céu)         | Ignis (Fogo)     | ix (Portador)   |
| Domin (Senhor)     | Tenebris (Trevas)| ens (Essência)  |
| Viri (Força)       | Mortem (Morte)   | os (Ligado a)   |
| Pax (Paz)          | Bellum (Guerra)  | arum (Herdeiro) |
| Aqua (Água)        | Vita (Vida)      | ana (Sagrado)   |
| Terra (Terra)      | Lumina (Luz)     | ion (Criador)   |


## Istruções para formar nomes:
1. Estrutura de Nomes: Um nome é formado pela combinação de até três elementos:
    - Prefixo: Representa uma ideia principal, como uma característica ou essência (ex.: Aure - Dourado).
    - Raiz: Representa o núcleo semântico do nome, frequentemente relacionado a um conceito central (ex.: Ignis - Fogo).
    - Sufixo: Indica um atributo ou papel, muitas vezes definindo a função ou estado do personagem (ex.: arius - Guardião).

2. Formas de Combinação: O modelo deve combinar os elementos seguindo estas estruturas ou variantes:
    - Prefixo + Raiz + Sufixo. Exemplo: Auremagnusius (Homem Dourado e Grande).
    - Raiz + Sufixo + Prefixo. Exemplo: IgnisariusCael (Guardião do Fogo do Céu).
    - Prefixo + Sufixo + Raiz. Exemplo: DominensTenebris (Essência do Senhor das Trevas).
    - Raiz + Prefixo + Sufixo. Exemplo: MortemVirius (Homem de Força e Morte).

3. Regras para a Criação de Nomes: 
    - Os elementos podem ser combinados livremente em qualquer ordem para criar nomes estilizados, desde que mantenham um significado coerente.
    - É permitido modificar ligeiramente as terminações para melhorar a fluidez do nome (ex.: mudar "us" para "um").
    - O significado resultante do nome deve ser inspirado pelas palavras originais e refletir um conceito que seja adequado a personagens de RPG, como guerreiros, magos ou entidades sobrenaturais.

4. Atribua a cada um do personagens apenas um dos seguintes papeis: {personagens_descrisao} 



## Saída Esperada:
"""

personagens_schema = (
"Você deve responder em formato JSON seguindo o seguinte esquema:\n"
"{\n"
"  \"characters\": [\n"
"    {\n"
"      \"name\": \"nome para o personagem com base nas **Instruções para formar nomes**.\",\n"
"      \"role\": \" Um dos papeis informados.\",\n"
"      \"motivation\": \"Detalhe o que move o personagem em relação à história principal, como vingança, ambição, dever, ou sobrevivência.\",\n"
"      \"personality\": \"Descreva os traços principais de comportamento e atitudes do personagem.\",\n"
"      \"connection_tram\": \"Especifique como o personagem está ligado aos eventos ou objetivos principais da aventura.\",\n"
"    }\n"
"  ]\n"
"}"
)

class Character(BaseModel):
    name: str
    role: str
    motivation: str
    personality: str
    connection_tram: str

class Characters(BaseModel):
    characters: List[Character]

personagens_cenas = model_response(system_prompt, promtpt, personagens_schema, Characters)

personagens_cenas.model_dump()

In [ ]:
system_prompt = "Você é um especialista em criação de histórias para RPG."

lugares = ""
atos = ["Introdução", "Desenvolvimento", "Clímax Preparatório", "Desfecho"]
i = 0 # indice para a cena
j=0
for local in story.locations:
    lugares += f"* {atos[i]}: {local.name} ({cenas[0][i]}) "
    
    # Procura por personagem no local i
    for even, personagem in cenas[1]:
        if atos[i] == even:
            if "ntagonista" not in personagens_cenas.characters[j].role:
                lugares += f"""onde você encontra {personagens_cenas.characters[j].name}, {personagens_cenas.characters[j].role}: {personagens_cenas.characters[j].connection_tram}"""
                j += 1
            
    # Procura por evento no local i
    for even, evento in cenas[2]:
        if even == atos[i]:
            if i ==3:
                lugares += f"onde {evento} e onde você encontra {personagens_cenas.characters[j].name}, {personagens_cenas.characters[j].role}: {personagens_cenas.characters[j].connection_tram}"
            else:
                lugares += f"onde {evento}"
    lugares += "\n"
    i += 1

nome_locais = ""
for local in story.locations:
   nome_locais += f"{local.name}; "

promtpt = f"""Para cada um dos seguintes locais: {nome_locais}, defina as seguintes informações detalhadas:

## Introdução:
1. **Ambientação**:   
{story.messages[0].content}

2. **O que aconteceu**: 
{story.messages[1].content}

3. **Como resolver**: 
{story.messages[2].content}

4. **Consequências de Falhar**:  
{story.messages[3].content}

5. **Lugares**:
{lugares}

## Instruções:
1. Os locais {nome_locais} devem estar presentes obrigatoriamente.
2. **Descrição do Local** inclua elementos visuais (o que se vê), auditivos (sons predominantes) e atmosféricos (sensação ou emoção gerada pelo ambiente).
3. **Papel do Local na Aventura**, explique o que ira acontecer neste local e como esse acontecimento contribui para a progressão da história de acordo com a cena em que está localizado:
    - Introdução: Nesta cena apresenta o contexto inicial, o conflito ou informações essenciais para começar a aventura.
    - Desenvolvimento: Nesta cena ajuda a avançar a trama inicial, como lugares que abrigam aliados, inimigos ou desafios importantes.
    - Clímax Preparatório: Nesta cena aumenta a tensão ou preparam os heróis para o confronto final.
    - Desfecho: Nesta cena tem o confronto final ou da resolução da aventura. Certifique-se de que pelo menos um dos locais seja designado para esta cena.

## Saída Esperada:
"""

locations_schema = (
"Você deve responder em formato JSON seguindo o seguinte esquema:\n"
"{\n"
"  \"locations\": [\n"
"    {\n"
"      \"name\": \"Informe apenas o nome marcante para o local.\",\n"
"      \"role\": \"Uma descrição detalhada do principal acontecimento no local.\",\n"
"      \"description\": \"Descrição detalhada, com elementos visuais, auditivos e atmosféricos do local\",\n"
"    }\n"
"  ]\n"
"}"
)

class Scene(BaseModel):
    name: str
    purpose: str

class Location(BaseModel):
    name: str
    role: str
    description: str

class Locations(BaseModel):
    locations: List[Location]

locais_cenas = model_response(system_prompt, promtpt, locations_schema, Locations)
print(f"Total de lugares: {len(locais_cenas.locations)} - {nome_locais}\n")

for local in locais_cenas.locations:
    print(f"{local.name}")
    print(f"    - Papel: {local.role}")
    print(f"    - Descrição: {local.description}")

In [ ]:
system_prompt = "Você é um especialista em criação de histórias para RPG."

locais_das_cenas = ""
atos = ["Introdução", "Desenvolvimento", "Clímax Preparatório", "Desfecho"]
i=0
for locais in locais_cenas.locations:
    locais_das_cenas+= f"* {atos[i]}: {locais.name}, {locais.role} É {locais.description}\n"
    i+=1
    
lista_pessoas = ""
i = 0 # indice para a cena
for character in personagens_cenas.characters:
    lista_pessoas += f"* {atos[i]}: {character.name} - {character.role}: {character.connection_tram}" 
    lista_pessoas += "\n"
    i += 1
promtpt = f"""Divida o objetivo principal da aventura: {story.objective}.

## Introdução: # 
1. **O que aconteceu**: 
{story.messages[1].content}

2. **Como resolver**: 
{story.messages[2].content}

3. **Objetivo Principal**:
{story.objective}

4. **Locais Importantes**:
{locais_das_cenas}

5. **Personagens Importantes**:
{lista_pessoas}

## Instruções:
1. Divida o objetivo principal em 4 cenas: Introdução, Desenvolvimento, Clímax Preparatório e Desfecho.
2. Para cada cena, defina:
    - Objetivo da Cena: O que os jogadores devem alcançar ou fazer.
    - Como realizar o objetivo: Como os jogadores podem cumprir a tarefa ou desafio da cena.
    - Obstáculos: Quais desafios ou dificuldades eles enfrentarão.
    - Conexão com a Próxima Cena: Como a cena leva à próxima.
3. Mantenha a progressão da narrativa lógica, garantindo que cada cena avance para a resolução final.
4. Seja direto e claro, explicando a conexão entre as cenas.

## Saída Esperada:"""

objective_schema = (
"Você deve responder em formato JSON seguindo o seguinte esquema:\n"
"{\n"
"  \"objectives\": [\n"
"    {\"scenes\":  \"Introdução\", \"objective\": \"Informe brevemente o objetivo para a cena, começando com: 'O objetivo dessa cena é ...'\", \"description\": \"uma breve descrição, de como realiza o objetivo.\", \"Connection_next_scene\": \"Descrição detalhada, sobre a conexão da cena atual com a proxima cena.\"}\n"
"    {\"scenes\":  \"Desenvolvimento\", \"objective\": \"Informe brevemente o objetivo para a cena, começando com: 'O objetivo dessa cena é ...'\", \"description\": \"uma breve descrição, de como realiza o objetivo.\", \"Connection_next_scene\": \"Descrição detalhada, sobre a conexão da cena atual com a proxima cena.\"}\n"
"    {\"scenes\":  \"Clímax Preparatório\", \"objective\": \"Informe brevemente o objetivo para a cena, começando com: 'O objetivo dessa cena é ...'\", \"description\": \"uma breve descrição, de como realiza o objetivo.\", \"Connection_next_scene\": \"Descrição detalhada, sobre a conexão da cena atual com a proxima cena.\"}\n"
"    {\"scenes\":  \"Desfecho\", \"objective\": \"Informe brevemente o objetivo para a cena, começando com: 'O objetivo dessa cena é ...'\", \"description\": \"uma breve descrição, de como realiza o objetivo.\", \"Connection_next_scene\": \"Descrição detalhada, sobre a conexão da cena atual com a proxima cena.\"}\n"
"  ]\n"
"}"
)

class objective(BaseModel):
    scenes: str
    objective: str
    description: str
    Connection_next_scene: str
    
class Objectives(BaseModel):
    objectives: List[objective]

res = model_response(system_prompt, promtpt, objective_schema, Objectives)
res.model_dump()

In [ ]:
for objetivo in res.objectives:
    print(objetivo.objective)

In [ ]:
print(story.objective)
print()
for objetivo in res.objectives:
    print(objetivo.Connection_next_scene)

Divisão dos arcos da aventura

In [168]:
# Prepara os pontos basicos dos 4 arcos
lugares = ""
for local in introducao.locations:
   lugares += f"* {local.name} ({local.purpose}): {local.evento}\n"

personagens = ""
for personagem in introducao.personagens:
   personagens += f"* {personagem.name}: {personagem.role}\n"


prompt_arco = f"""Sua tarefa é ajudar a criar narrativas divididas em 4 cenas claros e progressivos, utilizando os dados fornecidos em **Introdução (Dividida em Partes)** e seguindo as diretrizes a seguir:


## Introdução (Dividida em Partes): 
1. **Ambientação**:   
{introducao.messages[0].content}

2. **O que aconteceu**: 
{introducao.messages[1].content}

3. **Como resolver**: 
{introducao.messages[2].content}

4. **Consequências de Falhar**:  
{introducao.messages[3].content}

5. **Objetivo principal**:  
{introducao.objective}

## Locais Importantes:
{lugares}  

## Personagens Importantes:
{personagens}  

## Instruções:  
1. **Seleção de Locais**: Para cada cena selecione o loclal onde ela ira ocorrer. 
   - Priorize os **Locais Importantes** .  
   - Varie os tipos de locais entre os cenas para criar uma progressão interessante.  

2. **Seleção de Personagens**: Para cada cena selecione um ou mais personagem que irá participar.  
   - Priorize os **Personagens Importantes**. Caso não haja personagens definidos, crie personagens que façam sentido na história. 

3. **Seleção de Eventos**: Para cada cena selecione um evento que ocorra nesse local. 
   - Priorize os evento sugeridos para os locais. Caso não haja escolha eventos que promovam tensão narrativa e reflitam a progressão do arco.  
   - Certifique-se de que os eventos conectem-se diretamente ao objetivo principal.  
   - Relacione os eventos às partes da introdução, conforme indicado abaixo.  

4. **Definição de Objetivos**: Para cada cena defina um objetivo específico. Que seja parte do **objetivo principal**.  
   - Divida o objetivo principal em etapas claras e progressivas ao longo dos 4 cenas.  
   - Cada objetivo deve levar à próxima fase da história, culminando na resolução da trama no arco final.  
   - Relacione os objetivos ao local, personagem e evento do arco.  

5. **Conexão Geral**: Conecte as escolhas para criar um narrativa coesa. 
   - Certifique-se de que as escolhas de locais, personagens, eventos e objetivos conectem-se logicamente e criem uma narrativa coesa. 
   - Certifique-se de que cada arco introduza elementos ou personagens que contribuam diretamente para os eventos ou resoluções nos cenas seguintes, criando uma progressão temática clara e coesa.
   - Escale a tensão e a complexidade dos cenas à medida que a história avança, culminando em um clímax na Cena 4.  

## Saída Esperada:  
"""

arcos_schema = (
"Você deve responder em formato JSON seguindo o seguinte esquema:\n"
"{\n"
"  \"cenas\": [\n"
"    {\n"
"      \"name\": \"Cena 1: Introdução\",\n"
"      \"location\": \"Defina o local onde a cena inicial ocorre, refletindo a ambientação descrita na introdução.\",\n"
"      \"character\": \"Liste os personagens principais apresentados nesse momento, geralmente aqueles que introduzem o conflito ou contexto.\",\n"
"      \"event\": \"Descreva o cenário inicial e o incidente que dá início à aventura, alinhando-se com a 'Ambientação' e 'O que aconteceu' na introdução.\",\n"
"      \"objective\": \"Apresente o primeiro desafio ou mistério que os jogadores precisam compreender para seguir na história.\",\n"
"    },\n"
"    {\n"
"      \"name\": \"Cena 2: Obstáculo Inicial\",\n"
"      \"location\": \"Defina o local onde ocorre o primeiro grande desafio, conectado ao objetivo de encontrar o artefato ou avançar na missão.\",\n"
"      \"character\": \"Inclua aliados ou inimigos que os jogadores encontram, com foco em desenvolver a narrativa inicial.\",\n"
"      \"event\": \"Explique o primeiro grande obstáculo e como ele reflete os passos iniciais para resolver o problema principal, baseado em 'Como resolver'.\",\n"
"      \"objective\": \"Estabeleça o que os jogadores precisam superar nesse arco para seguir na história, como encontrar informações ou superar uma ameaça inicial.\"\n"
"    },\n"
"    {\n"
"      \"name\": \"Cena 3: Escalada\",\n"
"      \"location\": \"Descreva o local onde os eventos se tornam mais complexos ou perigosos, refletindo a progressão da história.\",\n"
"      \"character\": \"Apresente personagens que aumentam a tensão da trama, como inimigos mais poderosos ou aliados em risco.\",\n"
"      \"event\": \"Detalhe como o conflito se intensifica, ligando aos elementos de 'Como resolver' e 'Consequências de Falhar'.\",\n"
"      \"objective\": \"Defina o que os jogadores precisam realizar para evitar grandes perdas e preparar-se para o clímax da aventura.\",\n"
"    },\n"
"    {\n"
"      \"name\": \"Cena 4: Clímax e Resolução\",\n"
"      \"location\": \"Descreva o local onde acontece o confronto final ou a resolução da história, conectando ao objetivo principal.\",\n"
"      \"character\": \"Inclua o antagonista principal ou aliados cruciais que desempenham papéis essenciais no desfecho.\",\n"
"      \"event\": \"Explique o confronto ou evento final, com foco nas 'Consequências de Falhar' e no 'Objetivo principal'.\",\n"
"      \"objective\": \"Estabeleça o que os jogadores precisam alcançar para concluir a aventura com sucesso, como derrotar o vilão ou proteger o artefato.\",\n"
"    }\n"
"  ]\n"
"}"
)

class Cena(BaseModel):
   name: str
   location: str
   character: str
   event: str
   objective: str

class Cenas(BaseModel):
   cenas: List[Cena]


system_prompt = """Você é um especialista em criação de histórias para RPG. Siga as instruções fornecidas e responda em formato JSON seguindo o esquema especificado."""

In [ ]:
base_arcos = model_response(system_prompt, prompt_arco, arcos_schema, Cenas)
base_arcos.model_dump()

In [ ]:
print(prompt_arco)

Criação das cenas

In [128]:
## Cena 1: Introdução e Contexto
introducao_completa = texto_introducao
local_evento_atual = "Uma pequena vila e envolve Um grande mistério"
proximo_local_evento = "Um grande reino e envolve O Vilão ataca"
# {local} e envolve {evento}

prompt_sequencia_narracao = f"""### Contexto da Aventura
{introducao_completa}

### Tipo de Cena
Esta cena apresenta o cenário, o contexto e o gancho principal da história. Seu objetivo é colocar o personagem em movimento, mostrando a crise que ele deve resolver e respondendo a perguntas como: O que está em jogo? Quem ou o que causa o problema? Qual é o objetivo inicial do personagem?  

### Instruções 
1. Priorize **locais, eventos e personagens** mencionados na introdução.  
2. Caso a introdução não forneça informações suficientes, utilize as sugestões abaixo como fallback:  
   - **Local  e Evento Atual**: {local_evento_atual}  
   - **Próximo Local e Evento**: {proximo_local_evento} 
3. **Twists e Obstáculos**: Sugira até dois twists ou obstáculos inesperados para aumentar a tensão e manter o interesse.
4. Seja consistente e coerente ao definir o objetivo da cena atual e o objetivo para o proximo local.
5. **Extração de Informações** extraia exclusivaemente do Contexto da Aventura os seguintes dados:
  - **Itens importantes**: Informe o nome, função e condição de uso para os itens importantes para a trama principal e o seu status: "Não encontrado", "Não utilizado" e "Usado".
  - **Personagens importantes**: Informe o nome, papel e motivações para os personagens para a trama principal e o se sua participação ja foi concluida: "Não apareceu", "Não concluido" e "Concluido".
  - **Assuntos importantes**: Informe o nome, a da descrisão dos assuntos importantes para a trama principal e status se ele ja foi ou não tratado: "Não resonvido", "resonvido".

**Nota**: Sempre siga o tipo de cena descrito no contexto para manter o foco narrativo correto.
### Saida
"""
narrativa_schema = (
    "Você deve responder em formato JSON seguindo o seguinte esquema:\n"
    "{\n"
    "  \"name_locale\": \"Nome do local\",\n"
    "  \"scene_description\": \"Descreva o local com detalhes sensoriais e apresente o evento principal. Foque em criar atmosfera e introduzir conflitos ou desafios.\",\n"
    "  \"objetivo_description\": \"Explique o objetivo do personagem com uma ação iniciada no verbo no infinitivo\",\n"
    "  \"Connection_next_location\": \"Indique o próximo local no local atual com uma ação no imperativo e introduza um objetivo para o próximo local\",\n"
    "  \"itens\": [\n"
    "    {\"name\": \"Nome do item\", \"function\": \"Função do item\", \"condition_use\": \"Condição de uso do item\", \"status\": \"Status do item\"}\n"
    "  ],\n"
    "  \"personagens\": [\n"
    "    {\"name\": \"Nome do personagem\", \"role\": \"Papel do personagem\", \"motivation\": \"Motivação/segredos do personagem\", \"participation\": \"Seu participação ja foi concluida\"}\n"
    "  ],\n"
    "  \"assuntos\": [\n"
    "    {\"name\": \"Nome para o assunto\", \"description\": \"Breve explicação sobre o assunto\", \"status\": \"Status do assunto\"}\n"
    "  ],\n"
    "  \"twists\": [\n"
    "    {\"description\": \"Descrição da reviravolta\"}\n"
    "  ]\n"
    "}"
)

In [107]:
# Cena 2: Continuação da Cena Anterior
introducao_completa = texto_introducao
objetivo_anterior = "Investigar as mortes e desaparecimentos" # Objetivo da cena anterior (foi concuido)
local_evento_atual = "Partir em direção às Montanhas de Ardesia em busca dos Símbolos da Divindade" #(novo objetivo)


prompt_sequencia_narracao = f"""### Contexto da Aventura
{introducao_completa}

### Próxima Etapa da Jornada
Após {objetivo_anterior}, a história avança para {local_evento_atual}, onde um novo desafio ou descoberta impede o personagem de alcançar seu objetivo diretamente.

### Instruções
1. Respeite os **objetivos definidos**:
   - O **objetivo atual** deve ser cumprido **neste local** e não deve ser alterado ou substituído.
   - A **conexão com o próximo local** deve apresentar uma **ação nova** que avança a trama, sem repetir o objetivo atual.
   - A conexão com o próximo local deve ser independente e não pode interferir ou impedir a conclusão do objetivo atual.
2. **Local e Evento**:
   - O local deve ser descrito com **detalhes sensoriais** (visuais, sons, cheiros) para criar uma atmosfera imersiva.
   - O evento principal deve introduzir personagens, obstáculos ou conflitos que desafiam o personagem a cumprir seu objetivo.
3. **Twists e Obstáculos**:
   - Sugira até dois twists ou obstáculos inesperados que dificultem ou compliquem a jornada do personagem.

**Nota**: Sempre siga o tipo de cena descrito no contexto para manter o foco narrativo correto.
### Saida
"""

narrativa_schema = (
   "Você deve responder em formato JSON seguindo o seguinte esquema:\n"
   "{\n"
   "  \"name_locale\":  \"Nome do local\",\n"
   "  \"scene_description\": \"Descreva o local com detalhes sensoriais e apresente o evento principal. Foque em criar atmosfera e introduzir conflitos ou desafios.\"\n,"
   "  \"objetivo_description\": \"Descreva o objetivo do personagem no local atual, usando o verbo no infinitivo. NÃO mude o objetivo fornecido.\"\n,"
   "  \"Connection_next_location\": \"Apresente uma ação nova para o próximo local, evitando repetir o objetivo atual. A conexão deve avançar a trama com um novo desafio ou descoberta.\"\n,"
   "  \"twists\": [\n"
   "    {\n"
   "      \"description\": \"Descrição da reviravolta\"\n"
   "    },\n"
   "  ]\n"
   "}"
)

In [110]:
# Cena 3: Continuação da Cena Anterior para o clímax
introducao_completa = texto_introducao
objetivo_anterior = "Investigar o local e encontrar pistas sobre os Símbolos da Divindade" # Objetivo da cena anterior (foi concuido)
local_evento_atual = "Solicitar informações à figura encapuzada sobre os símbolos" #(novo objetivo)


prompt_sequencia_narracao = f"""### Contexto da Aventura
{introducao_completa}

### Próxima Etapa da Jornada
- Após cumprir o objetivo anterior: **{objetivo_anterior}**, o personagem agora deve avançar para: **{local_evento_atual}**.
- Esta cena deve preparar a transição direta para o **clímax da trama principal**, que será resolvido na próxima etapa.

### Instruções
1. **Objetivo da Cena**:
   - O personagem deve cumprir o **objetivo definido** nesta cena, sem modificá-lo ou desviá-lo: {local_evento_atual}.
   - A ação nesta cena deve ser **essencial** para avançar a história em direção ao clímax.

2. **Descrição do Local e Evento**:
   - Descreva o local com **detalhes sensoriais ricos** (visuais, sons, cheiros) para criar uma atmosfera imersiva.
   - Introduza um **evento principal** com personagens ou obstáculos que desafiem o personagem a cumprir o objetivo.

3. **Conexão com o Clímax**:
   - Apresente o **próximo local** com uma ação clara que leve diretamente ao **clímax da aventura**.
   - A conexão deve ser lógica e preparar a resolução da trama principal.
   - A conexão com a proxima cena deve ser diferente do objetivo anterior e objetivo atual

4. **Twists e Obstáculos**:
   - Sugira até **dois twists** ou obstáculos inesperados que aumentem a tensão, mas sem impedir a conclusão do objetivo atual.

**Nota**: Sempre siga o tipo de cena descrito no contexto para manter o foco narrativo correto.
### Saida
"""



narrativa_schema = (
   "Você deve responder em formato JSON seguindo o seguinte esquema:\n"
   "{\n"
   "  \"name_locale\":  \"Nome do local\",\n"
   "  \"scene_description\": \"Descreva o local com detalhes sensoriais e apresente o evento principal. Foque em criar atmosfera e introduzir conflitos ou desafios.\"\n,"
   "  \"objetivo_description\": \"Descreva o objetivo do personagem no local atual, usando o verbo no infinitivo. NÃO mude o objetivo fornecido.\"\n,"
   "  \"Connection_next_location\": \"Indique o próximo local com uma ação no imperativo e e conecte diretamente ao clímax da trama principal\"\n,"
   "  \"twists\": [\n"
   "    {\n"
   "      \"description\": \"Descrição da reviravolta\"\n"
   "    },\n"
   "  ]\n"
   "}"
)

In [114]:
# Cena 4: Final da Aventura
introducao_completa = texto_introducao
local_evento_atual = "Subir as escadas para encontrar o encontro com a figura encapuzada"
# {local} e envolve {evento}

prompt_sequencia_narracao = f"""### Contexto da Aventura
{introducao_completa}

### Cena Final: Desfecho da Aventura
Esta é a última cena da história, onde o personagem encara seu maior desafio e resolve (ou não) o problema central de forma definitiva. Não haverá novas cenas ou locais. O foco está no **clímax** e no encerramento da trama.

### Evento Final
A história chega ao seu ápice em {local_evento_atual}, onde o personagem enfrenta seu maior desafio. 
Esta é a última chance de resolver o problema principal de forma definitiva, determinando o destino da jornada e suas consequências.

### Instruções
1. **Local Atual**: Descreva o cenário com **detalhes sensoriais ricos** (visuais, sons, cheiros) para criar uma atmosfera tensa e imersiva, adequada ao confronto final.
2. **Evento Principal**: Apresente o **desafio decisivo** que o personagem enfrenta neste local. Inclua personagens envolvidos e a situação que leva ao desfecho.
3. **Objetivo do Personagem**: Descreva a **ação final** que o personagem deve realizar para resolver o conflito principal. Inicie a frase com o verbo no infinitivo.
4. **Conclusão e Consequências**: Resolva a história e explique as **consequências** das ações do personagem. Deixe claro se ele **alcança seu objetivo** ou não e quais são os impactos finais na trama.
5. **Twists Finais**: Sugira até **dois twists** ou revelações surpreendentes para tornar o final mais interessante, mas que não abram novos caminhos para a história.

**Importante**: 
- Esta é a **última cena** da aventura, e a história deve **terminar aqui**. Não deve haver conexão com um próximo local ou evento.
- O desfecho precisa resolver todos os pontos principais da trama.

### Saída
"""
narrativa_schema = (
   "Você deve responder em formato JSON seguindo o seguinte esquema:\n"
   "{\n"
   "  \"name_locale\":  \"Nome do local\",\n"
   "  \"scene_description\": \"Descreva o local com detalhes sensoriais e apresente o evento principal. Foque em criar atmosfera e introduzir conflitos ou desafios.\"\n,"
   "  \"objetivo_description\": \"Descreva o objetivo do personagem no local atual, usando o verbo no infinitivo. NÃO mude o objetivo fornecido.\"\n,"
   "  \"Connection_next_location\": \"Explique como a história termina e as consequências das ações do personagem.\"\n," # Na hora de passar isso para o modelo, isso deve ser chamado de concluisão. Não mudei o nome aui só para não precisar ceiar uma noca classe.
   "  \"twists\": [\n"
   "    {\n"
   "      \"description\": \"Descrição da reviravolta\"\n"
   "    },\n"
   "  ]\n"
   "}"
)

In [124]:
class Twist(BaseModel):
    description: str

class Personagem(BaseModel):
    name: str
    role: str
    motivation: str
    participation: str

class Item(BaseModel):
    name: str
    function: str
    condition_use: str
    status: str

class Assunto(BaseModel):
    name: str
    description: str
    status: str

class Location(BaseModel):
    name_locale: str
    scene_description: str
    objetivo_description: str
    Connection_next_location: str
    itens: List[Item]
    personagens: List[Personagem]
    assuntos: List[Assunto]
    twists: List[Twist]

In [ ]:
# Cena 1
local_cena_1 = base_arcos.arcos[0].local
personagem_cena_1 = base_arcos.arcos[0].personagem
evento_cena_1 = base_arcos.arcos[0].evento
objetivo_cena_1 = base_arcos.arcos[0].objetivos
local_proxima_cena = base_arcos.arcos[1].local
objetivo_proxima_cena = base_arcos.arcos[1].objetivos

prompt_cena = f"""Com base nos dados fornecidos, descreva a cena inicial da aventura de RPG que apresente o mundo, o contexto e motive os jogadores.
## Introdução da história: 
{introducao.messages[0].content} # apenas as duas partes relevantes para a cena 1
{introducao.messages[1].content}

## Local da Cena 1: 
{local_cena_1}

## Personagem da Cena 1:
{personagem_cena_1}

## Evento da Cena 1: 
{evento_cena_1}

## Objetivo da Cena 1:
{objetivo_cena_1}

## Local da Próxima Cena: 
{local_proxima_cena}

## Objetivo da Próxima Cena: (objetivo inicial que leva a esse lugar, ao chegar la pode acontecer alqo que leve a outro objetivo)
{objetivo_proxima_cena}

## Instruções:
1. Apresentar o cenário e o contexto do local atual.
2. Introduzir o personagem relacionado ao local ou à trama.
3. Descrever o evento significativo que ocorre no local, conectando-o ao objetivo principal.
4. Mostrar o gancho narrativo, destacando a crise ou desafio que o personagem deve resolver.
5. Fazer uma ligação natural para a próxima cena, mencionando o próximo local e objetivo.
6. Sugira até dois twists ou obstáculos inesperados para aumentar a tensão e manter o interesse.

## Saída Esperada:  
"""

cena_schema = (
    "Você deve responder em formato JSON seguindo o seguinte esquema:\n"
    "{\n"
    "  \"title\": \"Um titulo para a cena.\",\n"
    "  \"description\": \"Descreva os principais pontos que o Mestre devera usar para contruir a narrativa dessa cena. Um resumo do que pode acontecer do incio ao fim conectando o final dessa cena com o local e objetivo da proxima cena.\",\n"
    "  \"twists\": [\n"
    "    {\"description\": \"Descrição da reviravolta\"}\n"
    "  ]\n"
    "}"
)

class Twist(BaseModel):
    description: str

class Cena(BaseModel):
    title: str
    description: str
    twists: List[Twist]


"""
Cena:
    cena atual:
        resumo do que acontece nessa cena, quem ira fazer o que e onde vai fazer e porque vai fazer.
        o que pode acontecer (se der certo ou errado) e pra onde vai e porque vai ( tiver dado certo ou errado)
    proxima cena:
        local_objetivo: pra onde vai e o porque vai.
    twists:

"""

In [322]:
system_prompt = """Você é um criador de narrativas para RPG de mesa. Sua tarefa é descrever cenas para ajudar o Mestre de Jogo a narrar a aventura.
1. **Foco**: Apresente o cenário, o evento principal e os personagens envolvidos, conectando-os ao objetivo da cena.
2. **Protagonistas**: Os jogadores são os personagens principais, e os NPCs (personagens fornecidos) estão lá para apoiar ou desafiar a história.
3. **Clareza**: A descrição deve ser direta e evocativa, sem excesso de detalhes, mas suficiente para criar uma cena imersiva.
4. **Conexão**: Faça uma transição natural para a próxima cena, destacando o próximo local e objetivo.

"""

In [ ]:
print(prompt_cena)

In [ ]:
base_cena = model_response(system_prompt, prompt_cena, cena_schema, Cena)
base_cena.model_dump()
print(f"Titulo: {base_cena.title}")
print(f"{dividir_texto_em_linhas(base_cena.description, largura=100)}")
print(f"\nTwinst:")
for twinst in base_cena.twists:
    print(f"- {dividir_texto_em_linhas(twinst.description, largura=100)}")

Define a cena inicial (ve o que da pra simplificar, talvez de pra tirar os exemplos)

In [225]:
cena_inicial_cena = """Você é um chatbot narrador de aventuras de RPG de mesa, que usa o estilo de narração imersivo descritivo para uma descrição detalhada do ambiente, personagem ou evento com detalhes sensoriais (visão, som, cheiro, tato, etc.) e escreve as falas em uma nova linha com o estilo de trancrisção iniciando as falas com um hífen (—).
Crie uma introdução para a cena inicial em até duas mensagens, tendo como base a Introdução e Descrisão da cena.
Gere textos curtos, com poucas palavras.
Sempre finalize com um gancho claro, como: "O que vocês fazem?"
Informe o Status atual da narrativa, que pode ser: "Em andamento", "Finalizada".
Siga o formato das mensagens abaixo para criar a introdução da cena:
Exemplo 1
Mensagem 1: (uma curta descrição da Cena) 
A vila de Grenwald está em silêncio mortal. Telhados afundados, musgo escuro cobrindo as paredes e um cheiro de podridão pairando no ar. Você caminha firme, a armadura rangendo, enquanto observa as marcas de garras nas portas com olhos inquietos. Um vento frio sopra, trazendo uma sensação de que estão sendo observados.

Mensagem 2: (Ação ou fala do NPC)
Do beco, um velho caçador surge da sombra, o arco firme nas mãos. Ele olha diretamente para Você.
— "Não devia estar aqui."
Sua voz rouca ecoa, e ele aponta para as marcas de garras.
— "Ele passou por aqui há muito tempo... mas o mal nunca dorme."
O silêncio pesa. Você segura o cabo da espada. O que vocês fazem?

Exemplo 2: 
Mensagem 1: (uma curta descrição da Cena) 
A taverna "O Cangaceiro Vermelho" vibra com vozes embriagadas, cheiro de cerveja barata e lamparinas de óleo iluminando o ambiente. Vocês observam tudo de um canto, atento, enquanto um de vocês mexe em algumas sacolas deixadas para trás.

Mensagem 2: (Ação ou fala do NPC)
Um estrondo violento! A porta é arrombada, e homens em armaduras invadem a taverna.
— "Todo mundo no chão, agora!" — grita o líder dos invasores.
Clientes correm, cadeiras caem, e o cheiro de ferro e fumaça invade o salão. O que vocês fazem?"""

In [226]:
mensagens_schema = (
    "Você deve responder em formato JSON seguindo o seguinte esquema:\n"
    "{\n"
    "  \"title\": \"Título do cenário\",\n"
    "  \"messages\": [\n"
    "    {\"number\": 1, \"content\": \"Primeira mensagem\"},\n"
    "    {\"number\": 2, \"content\": \"Segunda mensagem\"}\n"
    "  ],\n"
    "  \"status\": \"Status atual da narrativa\"\n"
    "}"
)

In [227]:
class Message(BaseModel):
    number: int
    content: str

# Classe que representa o cenário completo
class Messages(BaseModel):
    title: str
    messages: List[Message]
    status: str

In [ ]:
# enables `response_model` in create call
client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",  # required, but unused
    ),
    mode=instructor.Mode.JSON,
)


nome_modelo = lista_modelos[modelo_id]
print(f"{nome_modelo}\n")
narrativa = client.chat.completions.create(
    model=nome_modelo,
    messages=[
        {
            "role": "system",
            "content": f"{cena_inicial_cena}\n\n{mensagens_schema}"
        },
        {   "role": "user",
            "content": f"Introdução:\n{texto_introducao[0]}\n{texto_introducao[1]}\n{texto_introducao[2]}\n{texto_introducao[3]}\n\Twist para a aventura:\n{lista_twist}\n\nDescrisão da cena:\n{lista_cenas[0]}"
        }
    ],
    response_model= Messages,
    extra_body={"gpu_layers":-1, "temperature": 0.7},
    
)


if True:
    print(f"{narrativa.title}")
    print()
    for mensagem in narrativa.messages:
        print(dividir_texto_em_linhas(mensagem.content, largura=100))
        print("=")
    print(f"\n{narrativa.status}")
else:
    print(narrativa.choices[0].message.content)

##### Construção do Resumo da aventura 

1. Contexto Geral da Aventura \
Este é o panorama inicial que orienta toda a história.

* Título da Aventura: Um nome impactante e temático.
* Tema e Gênero: Ex.: fantasia épica, horror gótico, steampunk.
* Resumo: Uma visão geral da aventura em 2-3 frases.
    * Exemplo: "Os jogadores precisam explorar ruínas antigas para encontrar um artefato mágico capaz de salvar o reino de uma praga devastadora."
* Tom e Estilo: Ex.: sombrio, heroico, cômico.

In [33]:
cenario = cenario_aleatorio() # Seleciona o cenário e prepara o prompt

system_prompt = """Você é um especialista em criação de histórias para RPG."""

promtpt_general_context = f"""Preencha os dados de saida com base na Descrição e instruções a seguir:

## Descrição:
{cenario}

## Instruções:
1. O theme principal como Ex.: fantasia épica, horror gótico, steampunk.
2. summary é um breve resumo da aventura, falando o que aconteceu e o o que precisa ser feito para resolver e as consequências caso falhe. 
    * Deve ser baseado em **Problema que aconteceu**, **O que precisa ser feito** e **Consequências**.
3. Style é o estilo da aventura, como Ex.: sombrio, heroico, cômico, etc.
4. Não inclua nomes especificos para personagens ou locais.
5. Deixe claro qual o problema que aconteceu, o que precisa ser feito e as consequências caso falhe em resolver o problema.
6. Seja coerente e consistente.

## Saida
Você deve responder em formato JSON seguindo o seguinte esquema:\n"""


schema_json_string = "{\n"
schema_json_string += "  \"title\": \"Um nome impactante e temático.\",\n"
schema_json_string += "  \"theme\": \"Um Tema para a aventura.\",\n"
schema_json_string += "  \"summary\": \"Uma visão geral da aventura em 3-4 frases.\",\n"
schema_json_string += "  \"style\": \"O Estilo da aventura.\"\n"
schema_json_string += "}\n"
general_context_schema = json.loads(schema_json_string)

class General_context(BaseModel):
    title: str
    theme: str
    summary: str
    style: str

general_context = model_response(system_prompt, promtpt_general_context, general_context_schema, General_context)

print(cenario)
print(f"\nTitulo: {general_context.title}")
print(f"Tema: {general_context.theme}")
print(f"Style: {general_context.style}")
print(f"Summary:\n{dividir_texto_em_linhas(general_context.summary, 100)}")

Descrisão do Cenário:
- Tema principal: Fantasy
- Problema que aconteceu: Um tirano dominou terras.
- O que precisa ser feito: Caçar um monstro. 
- Consequências: Uma guerra eterna acontecerá.
- Ambientação: (Épica) Em um reino onde a magia floresce e ruínas antigas sussurram segredos de eras passadas, você trilha seu caminho como um aventureiro, desbravando masmorras sombrias e enfrentando feras lendárias em busca de glória e tesouros esquecidos.

Titulo: A Era do Tirano
Tema: Fantasia Épica
Style: Heroico e Aventuroso
Summary:
Um tirano dominou as terras, levando a uma era de opressão e medo. Para restaurar a justiça e a
liberdade, é necessário caçar um monstro lendário que responde ao tirano. Se falhar, uma guerra
eterna irá dizimar a terra e destruir as últimas sementes da esperança.


Definição dos nomes dos locais e personagens.

In [14]:
cenas = cenas_aleatorias()

def formatar_dados(dados):
    lista_tipo = lista_lugares = ""
    for parte, detalhes in dados.items():
        personagem = detalhes.get('personagem')
        lugar = detalhes.get('lugar')

        if personagem:  # Verifica se personagem não é None
            lista_tipo +=f" {personagem};"
        lista_lugares +=f" {lugar};"
    lista_lugares +=f" um grande reino;"
    lista_lugares +=f" uma vila pequena."
    return lista_tipo, lista_lugares

cenas

{'Introduction': {'lugar': 'uma taverna movimentada',
  'evento': None,
  'personagem': 'um familiar de um mago'},
 'Development': {'lugar': 'uma viela escura',
  'evento': 'Monstros errantes aparecem',
  'personagem': None},
 'Preparatory Climax': {'lugar': 'um acampamento',
  'evento': None,
  'personagem': 'um fantasma ou desmorto'},
 'Climax': {'lugar': 'uma tribo selvagem',
  'evento': 'Você encontra um tesouro',
  'personagem': None}}

In [19]:
# Modelo base para um local
def create_location(type):
    return {
        "first_word": "Primeira palavra do nome do local, representando a localização (Localização), conforme as **Instruções**.",
        "second_word": "Segunda palavra do nome do local, combinando Prefixo + Raiz + (opcionalmente Sufixo), conforme as **Instruções**.",
        "type": type,
    }

def create_character(role):
    return {
        "first_name": "Primeira palavra do nome do personagem, combinando Prefixo + Raiz, conforme as **Instruções**.",
        "last_name": "Segunda palavra do nome do personagem, combinando Raiz + Sufixo, conforme as **Instruções**",
        "role": role,
    }
locations = []
characters = []

for ato in ['Introduction', 'Development', 'Preparatory Climax', 'Climax']:
    if cenas[ato]['personagem'] != None:
        characters.append(create_character(cenas[ato]['personagem']))
    locations.append(create_location(cenas[ato]['lugar']))


locations.append(create_location("um grande reino"))
locations.append(create_location("uma vila pequena"))
adventure_schema = {
    "characters": characters,
    "locations": locations,
}

# Converter para JSON formatado
names_schema = json.dumps(adventure_schema, indent=4, ensure_ascii=False)

In [24]:
# Preciso rever a escolha de locias, o modelo sempre gera nome para reinos, cidades pois é necessario para a historia e as cenas sorteadas nem sempre tem esses locais.
personagens_descrisao, lugares_descrisao = formatar_dados(cenas)

promtpt_names = f"""Crie uma lista com nomes para as seguintes roles of the characters e para os Types of places.
## Tabela de Elementos:

| **Prefixo** (Significado) | **Raiz** (Significado)  | **Sufixo** (Significado)  | **Localização** (Significado) |
|---------------------------|-------------------------|---------------------------|-------------------------------|
| Aure (Dourado)            | Magnus (Grande)         | ius (Homem)               | Castrum (Fortaleza)           |
| Luci (Luz)                | Noct (Noite)            | us (Ser)                  | Silva (Floresta)              |
| Cor (Coração)             | Fulg (Brilho)           | arius (Guardião)          | Portus (Porto)                |
| Flavi (Amarelo)           | Tempest (Tempestade)    | or (Aquele que)           | Via (Estrada)                 |
| Cael (Céu)                | Ignis (Fogo)            | ix (Portador)             | Cavernus (Caverna)            |
| Domin (Senhor)            | Tenebris (Trevas)       | ens (Essência)            | Castellum (Castelo)           |
| Viri (Força)              | Mortem (Morte)          | os (Ligado a)             | Oppidum (Vila)                |
| Pax (Paz)                 | Bellum (Guerra)         | arum (Herdeiro)           | Regnum (Reino)                |
| Aqua (Água)               | Vita (Vida)             | ana (Sagrado)             | Labyrinthus (Labirinto)       |
| Terra (Terra)             | Lumina (Luz)            | ion (Criador)             | Templum (Templo)              |
| Umbra (Sombra)            | Glacius (Gelo)          | eum (Lugar)               | Taberna (Taverna)             |
| Ignia (Chama)             | Ventus (Vento)          | um (Refúgio)              | Arca (Veículo)                |
| Vita (Vida)               | Nex (Morte)             | orium (Reino)             | Planus (Plano)                |
| Tempora (Tempo)           | Statera (Equilíbrio)    | or (Aquele que)           | Spelunca (Masmorra)           |
| Altus (Alto)              | Frigus (Frio)           | inis (Guardião)           | Desolatus (Lugar ermo)        |
| Ignis (Fogo)              | Serenus (Calmo)         | itas (Estado de ser)      | Angiportus (Viela)            |
| Solis (Sol)               | Ferox (Feroz)           | ilis (Capaz de)           | Iter (Estrada)                |
| Lunaris (Lua)             | Venustus (Belo)         | atus (Marcado por)        | Urbs (Cidade)                 |


## Instruções:
1. Crie nomes que estejam relacionados ao type of places and a role of the characters informados.
2. Crie nomes para as seguintes roles of the characters: {personagens_descrisao}
3. Para os personagens, crie nomes com duas palavras:
  - first_name deve combinar Prefixo + Raiz.
  - last_name deve combinar Raiz + Sufixo.
  - Exemplo: Lucimagnus Ignisius (Grande luz do fogo).

4. Crie nomes para os seguintes Types of places: {lugares_descrisao}
  - first_word deve ser coerente o tipo de lugar. Ex.: Oppidum (Vila), Planus (Plano).
  - second_word deve combinar Prefixo + Raiz + (opcionalmente Sufixo).
  - Exemplo: Castrum Auremagnus (fortaleza dourada).

## Saída:
Você deve responder em formato JSON seguindo o seguinte esquema:\n"""

names_schema = """
{
  "world_name": "Nome para o mundo onde a aventura acontece. Com base na Tabela de Elementos.",
  "characters": [
    {
      "first_name": "Primeira palavra do nome do personagem.",
      "last_name": "Segunda palavra do nome do personagem.",
      "role": "Um dos Role of the characters informados."
    }
  ]
  "locations": [
    {
      "first_word": "Primeira palavra do nome do local, representando a localização.",
      "second_word": "Segunda palavra do nome do local.",
      "type": "Um dos Types of places informados."
    }
  ]
}
"""
#names_schemas = json.loads(names_schema)

class Character(BaseModel):
    first_name: str
    last_name: str
    role: str

class Location(BaseModel):
    first_word: str
    second_word: str
    type: str

class NameGenerator(BaseModel):
    world_name: str 
    characters: List[Character]
    locations: List[Location]

system_prompt = "Você é um Dungeon Master especialista em criação de nomes para RPG."


personagens_cenas = model_response(system_prompt, promtpt_names, names_schema, NameGenerator) # Pode não gerar nome para todos
print(f"locais: {len(personagens_cenas.locations)}{lugares_descrisao}")
print(f"personagens: {len(personagens_cenas.characters)}{personagens_descrisao}")
personagens_cenas.model_dump() # um nome para o mundo

locais: 6 uma taverna movimentada; uma viela escura; um acampamento; uma tribo selvagem; um grande reino; uma vila pequena.
personagens: 2 um familiar de um mago; um fantasma ou desmorto;


{'world_name': 'Magnavigem',
 'characters': [{'first_name': 'Lucimagnus',
   'last_name': 'Ignisius',
   'role': 'um familiar de um mago'},
  {'first_name': 'Ignius',
   'last_name': 'Tenebrious',
   'role': 'um fantasma ou desmorto'}],
 'locations': [{'first_word': 'Taberna',
   'second_word': 'Lucianoxius',
   'type': 'taverna movimentada'},
  {'first_word': 'Angiportus',
   'second_word': 'Dominius',
   'type': 'viela escura'},
  {'first_word': 'Castrum',
   'second_word': 'Altusmagnus',
   'type': 'acampamento'},
  {'first_word': 'Spelunca',
   'second_word': 'Corbrilius',
   'type': 'tribo selvagem'},
  {'first_word': 'Regnum', 'second_word': 'Paxmagnus', 'type': 'grande reino'},
  {'first_word': 'Oppidum',
   'second_word': 'Corfolgus',
   'type': 'vila pequena'}]}

2. Introdução
Aqui você define as bases para os jogadores entenderem o mundo e a trama. Divida em partes se for uma campanha longa:

* Parte 1: O Mundo Antes do Problema
    * Descrição Geral: Como era o mundo antes do evento principal?
        * Exemplo: "O reino de Eldoria era pacífico, com aldeias prosperando sob a proteção de uma floresta mágica."
    * Detalhes Relevantes:
        * Reinos, cidades ou facções importantes.
        * História ou lendas que conectam ao enredo.
* Parte 2: O Problema
    * O Evento Principal: O que deu início à aventura?
        * Exemplo: "Uma praga mágica começou a transformar a flora em cinzas, colocando o reino em risco."
    * Causas e Mistérios Iniciais: Dicas sobre o que pode estar causando o problema.

* Parte 3: O que Precisa Ser Feito 

    * Exemplo: Para impedir o despertar de Zal'thorix, os heróis devem: 
        * Reunir fragmentos de um artefato lendário chamado "Cétus da Vida Eterna".
        * Purificar os santuários corrompidos espalhados pelo mundo.
        * Desafiar a própria Morte no Domínio dos Eternos para restaurar o ciclo natural.

* Parte 4: As Consequências
    * Sucesso: O mundo é salvo, o ciclo é restaurado, mas não sem um alto custo. Os heróis podem perder algo importante (vida, memórias ou conexões).
    * Falha: Zal'thorix desperta, trazendo um apocalipse de trevas eternas. O mundo se torna um deserto desolado de mortos e espectros.

In [25]:
locais_importantes = ""
for locais in personagens_cenas.locations:
  locais_importantes += f" * Em {locais.type} chamado {locais.first_word} {locais.second_word}.\n"

contexto_geral = f"""Titulo: {general_context.title}
Tema: {general_context.theme}
Style: {general_context.style}
Locais importantes:\n{locais_importantes}
Summary:\n{dividir_texto_em_linhas(general_context.summary, 100)}"""

prompt_introducao = f"""## Contexto geral:
{contexto_geral}

## Instruções:
1. Descreva como era o mundo antes do evento principal.
2. Inclua dois detalhes relevantes sobre a história, reinos, cidades, lendas ou facções conectadas à trama principal.
3. Explique o problema principal que iniciou a aventura e forneça duas pistas sobre suas causas.
4. Não inclua nomes específicos para personagens.
5. Explique as consequências em caso de sucesso ou falha na aventura.
6. Crie o main_objective com base no contexto fornecido.
7. Use os nomes dos locais fornecidos.

**Importante:** Seja claro e consistente.

## Saída esperada:
Você deve responder em formato JSON seguindo o esquema:\n"""

schema_json_string_intro = """
{
  "pre_event_world": {
    "general_description": "Descrição geral do mundo antes do evento principal.",
    "relevant_details": [
      "Detalhe relevante 1 sobre o mundo.",
      "Detalhe relevante 2 sobre o mundo.",
      "Detalhe relevante 3 sobre o mundo."
    ]
  },
  "objective": {
    "main_objective": "Descrição do objetivo principal que impulsiona a aventura.",
    "clues": [
      "Pista 1 para resolver o problema.",
      "Pista 2 para resolver o problema.",
      "Pista 3 para resolver o problema."
    ]
  },
  "consequences": {
    "success": "Descrição das consequências em caso de sucesso.",
    "failure": "Descrição das consequências em caso de falha."
  }
}
"""
introductions_schema = json.loads(schema_json_string_intro)



class Consequences(BaseModel):
  success: str
  failure: str

class Objective(BaseModel):
  main_objective: str
  clues: List[str]

class Word(BaseModel):
  general_description: str
  relevant_details: List[str]

class Introductions(BaseModel):
  pre_event_world: Word
  objective: Objective
  consequences: Consequences

system_prompt = """Você é um Dungeon Master especialista em criação de histórias para RPG."""

introducao = model_response(system_prompt, prompt_introducao, introductions_schema, Introductions)
introducao.model_dump() 

{'pre_event_world': {'general_description': 'O mundo era um lugar de paz e prosperidade, onde as cidades prósperas e os reinos poderosos prosperavam lado a lado. A cultura e a tecnologia avançavam rapidamente, e as pessoas viviam em harmonia com a natureza.',
  'relevant_details': ['A Taberna Lucianoxius era um ponto de encontro favorito para viajantes e mercadores de todas as partes do mundo.',
   'O Reino de Paxmagnus era um dos mais poderosos do mundo, conhecido por suas vastas extensões de terra e seu exército invencível.']},
 'objective': {'main_objective': 'Encontrar a fonte das criaturas das trevas e impedir que elas destruam o mundo, salvando assim a humanidade e a beleza do mundo.',
  'clues': ['Uma fonte antiga diz que as criaturas das trevas são originadas do submundo, um lugar proibido para os mortais.',
   'Rumores não confirmados sugerem que o mal pode estar relacionado à destruição de um antigo equilíbrio entre o mundo e o submundo.']},
 'consequences': {'success': 'Se a

In [26]:
print(prompt_introducao)

## Contexto geral:
Titulo: Noite Eterna
Tema: Fantasia
Style: Sombrio
Locais importantes:
 * Em taverna movimentada chamado Taberna Lucianoxius.
 * Em viela escura chamado Angiportus Dominius.
 * Em acampamento chamado Castrum Altusmagnus.
 * Em tribo selvagem chamado Spelunca Corbrilius.
 * Em grande reino chamado Regnum Paxmagnus.
 * Em vila pequena chamado Oppidum Corfolgus.

Summary:
Criaturas das trevas começam a surgir à noite, ameaçando destruir o mundo se não forem detidas. Uma
grande viagem precisa ser empreendida para encontrar a solução para esse problema e evitar a
destruição do mundo.

## Instruções:
1. Descreva como era o mundo antes do evento principal.
2. Inclua dois detalhes relevantes sobre a história, reinos, cidades, lendas ou facções conectadas à trama principal.
3. Explique o problema principal que iniciou a aventura e forneça duas pistas sobre suas causas.
4. Não inclua nomes específicos para personagens.
5. Explique as consequências em caso de sucesso ou falha n

In [27]:
relevant_details = ""
for relevant_detail in introducao.pre_event_world.relevant_details:
  relevant_details+= f" * {relevant_detail}\n"

clues = ""
for clue in introducao.objective.clues:
    clues+= f" * {relevant_detail}\n"

locais = ""
i=0
for local in personagens_cenas.locations:
    locais+= f" * step {i+1}: Em {local.type} chamado {local.first_word} {local.second_word}.\n"
    i+=1

character_names= ""
for character in personagens_cenas.characters:
  character_names += f" * {character.role} chamado {character.first_name} {character.last_name}.\n"

contexto_geral = f"""Titulo: {general_context.title}
Tema: {general_context.theme}
Style: {general_context.style}
Main Objective: {introducao.objective.main_objective} 
Charaacter Names:\n{character_names}
Locais:\n{locais}
Summary:\n{dividir_texto_em_linhas(introducao.pre_event_world.general_description, 100)}

Relevant Details:\n{relevant_details}
Clues:\n{clues}
Consequences:\n  * Success {introducao.consequences.success}\n  * Failure {introducao.consequences.failure}"""



prompt_solution = f"""## Contexto geral:
{contexto_geral}

## Instruções:
1. Divida o objetivo principal em quatro etapas (uma para cada arco narrativo):
   - Arco 1: Apresentação e introdução do conflito inicial.
   - Arco 2: Primeira etapa da jornada, buscando informações, pistas ou recursos.
   - Arco 3: Escalada do conflito, enfrentando aliados do antagonista ou grandes obstáculos.
   - Arco 4: O clímax, com a resolução do problema principal.
2. Para cada etapa, forneça:
   - Descrição clara do objetivo principal de cada arco.
   - Dilemas ou escolhas relevantes para cada arco.
   - Impacto narrativo do sucesso ou fracasso no arco.
3. Não inclua nomes especificos para personagens ou locais.
4. Seja criativo ao expandir os eventos que ocorrerão durante a jornada.

**Importante:** Não repita informações já fornecidas em outras partes, como localização ou NPCs.

## Saída esperada:
Você deve responder em formato JSON seguindo o esquema:\n"""

schema_json_string_solution = """
{
  "solution": [
    {
      "step": 1,
      "description": "Descrição do objetivo do Arco 1.",
      "moral_dilemma": "Dilema ou escolha relevante no Arco 1.",
      "impact_on_story": "Impacto narrativo do sucesso ou fracasso no Arco 1."
    },
    {
      "step": 2,
      "description": "Descrição do objetivo do Arco 2.",
      "moral_dilemma": "Dilema ou escolha relevante no Arco 2.",
      "impact_on_story": "Impacto narrativo do sucesso ou fracasso no Arco 2."
    },
    {
      "step": 3,
      "description": "Descrição do objetivo do Arco 3.",
      "moral_dilemma": "Dilema ou escolha relevante no Arco 3.",
      "impact_on_story": "Impacto narrativo do sucesso ou fracasso no Arco 3."
    },
    {
      "step": 4,
      "description": "Descrição do objetivo do Arco 4.",
      "moral_dilemma": "Dilema ou escolha relevante no Arco 4.",
      "impact_on_story": "Impacto narrativo do sucesso ou fracasso no Arco 4."
    }
  ]
}
"""
solution_schema = json.loads(schema_json_string_solution)

class SolutionStep(BaseModel):
  step: int
  description: str
  moral_dilemma: str
  impact_on_story: str

class Solution(BaseModel):
  solution: List[SolutionStep]

system_prompt = """Você é um Dungeon Master especialista em criação de histórias para RPG."""

solutions = model_response(system_prompt, prompt_solution, solution_schema, Solution)
for step in solutions.solution:
  print(f"Arco {step.step}")
  print(f"{dividir_texto_em_linhas(step.description, 100)}\n")
  print(f"{dividir_texto_em_linhas(step.moral_dilemma, 100)}\n")
  print(f"{dividir_texto_em_linhas(step.impact_on_story, 100)}")
  print("="*100)

Arco 1
Apresentar o conflito inicial e introduzir os personagens principais em uma taverna movimentada.

Decidir se o grupo deve confiar em um estranho que oferece informações valiosas sobre as criaturas
das trevas.

Se o grupo confiar no estranho, eles recebem informações valiosas, mas também correm o risco de
serem enganados. Se não confiarem, eles precisam encontrar outras fontes de informações, o que pode
ser difícil.
Arco 2
Buscar informações e pistas em uma viela escura, onde os grupos de criaturas das trevas foram
vistos.

Decidir se o grupo deve avançar em uma viela escura perigosa ou voltar atrás e procurar uma rota
mais segura.

Se o grupo avançar em segurança, eles podem encontrar informações valiosas, mas também correm o
risco de serem atraídos pela criaturas das trevas. Se eles voltarem atrás, eles podem perder a
oportunidade de encontrar informações valiosas.
Arco 3
Enfrentar aliados do antagonista em um acampamento, onde os grupos de criaturas das trevas estão se
reunind

In [28]:
print(prompt_solution)

## Contexto geral:
Titulo: Noite Eterna
Tema: Fantasia
Style: Sombrio
Main Objective: Encontrar a fonte das criaturas das trevas e impedir que elas destruam o mundo, salvando assim a humanidade e a beleza do mundo. 
Charaacter Names:
 * um familiar de um mago chamado Lucimagnus Ignisius.
 * um fantasma ou desmorto chamado Ignius Tenebrious.

Locais:
 * step 1: Em taverna movimentada chamado Taberna Lucianoxius.
 * step 2: Em viela escura chamado Angiportus Dominius.
 * step 3: Em acampamento chamado Castrum Altusmagnus.
 * step 4: Em tribo selvagem chamado Spelunca Corbrilius.
 * step 5: Em grande reino chamado Regnum Paxmagnus.
 * step 6: Em vila pequena chamado Oppidum Corfolgus.

Summary:
O mundo era um lugar de paz e prosperidade, onde as cidades prósperas e os reinos poderosos
prosperavam lado a lado. A cultura e a tecnologia avançavam rapidamente, e as pessoas viviam em
harmonia com a natureza.

Relevant Details:
 * A Taberna Lucianoxius era um ponto de encontro favorito para via

3. Estrutura dos Arcos
Cada arco deve ter um foco claro e objetivos definidos para os jogadores.

* Arco 1: Introdução e Primeiros
    * Objetivo Principal: Apresentar o mundo e o problema aos jogadores.
* Arco 2: Primeira Etapa da Jornada
    * Objetivo Principal: Colocar os jogadores em movimento para resolver o problema.

* Arco 3: Aproximação do Clímax
    * Objetivo Principal: Escalar a tensão e preparar os jogadores para o final.

* Arco 4: O Clímax
    * Objetivo Principal: Resolver o problema principal.

In [29]:
Eventos = []
lugares = []
for ato in cenas.keys():
    if cenas[ato]['personagem'] != None:
        for persona in personagens_cenas.characters:
            if  cenas[ato]['personagem'] in persona.role:
                A = random.randrange(len(Assunto))
                Eventos.append(f"os jogadores encontram com {persona.role} chamado {persona.first_name} {persona.last_name}, relacionado ao tema: {Assunto[A]}")
                break
    else:
        Eventos.append(f"{cenas[ato]['evento']}")
    
    for lugar in personagens_cenas.locations:
        if  cenas[ato]['lugar'] in lugar.type:
            lugares.append(f"{lugar.type} chamado {lugar.first_word} {lugar.second_word}")
            break

print(f"Lugares: {len(lugares)}")
print(f"Eventos: {len(Eventos)}")

Lugares: 0
Eventos: 4


In [31]:
lista_arcos = []
for i in range(4):
    arco = ""
    if i ==3: # o ultimo arco
        arco += f"""## Arco Final (Clímax)\n{solutions.solution[i].description}\n{solutions.solution[i].moral_dilemma}\n{solutions.solution[i].impact_on_story}\n\n""" 
    else:
        arco += f"""## Arco Atual\n{solutions.solution[i].description}\n{solutions.solution[i].moral_dilemma}\n{solutions.solution[i].impact_on_story}\n\n""" 

    if i < 3: # o ultimo arco não tem proximo arco.
        arco += f"""## Próximo Arco\n{solutions.solution[i+1].description}\n{solutions.solution[i+1].moral_dilemma}\n{solutions.solution[i+1].impact_on_story}"""
    
    lista_arcos.append(arco)


lista_aventura = []
for arco_atual in range(4):
    print(f"Arco: {arco_atual+1}")
    prompt_arcos = f"""
    ## General Description:
    {introducao.pre_event_world.general_description}

    {lista_arcos[arco_atual]}    

    ## Instruções:

    1. Defina uma sequência de cenas para o arco atual. Cada cena deve ter um local e um ou mais eventos que desafiem os jogadores e os conduzam em direção ao objetivo central do arco.

    2. Para cada cena, descreva eventos ou encontros variados que desafiem os jogadores. Estes eventos podem ser de natureza física, social, exploratória ou estratégica.

    3. Especifique o objetivo central do arco, o que os jogadores devem alcançar. Qual é o principal conflito ou problema que será abordado nas cenas?

    4. Defina um problema ou desafio para os jogadores resolverem em cada cena. Isso deve estar alinhado ao objetivo geral do arco.

    5. O arco atual deve ser concluído com um evento marcante que conecte logicamente com o próximo arco. Este evento de transição deve preparar o terreno para o que virá a seguir.

    6. Escreva um resumo narrativo do arco atual, detalhando:
        * As ações dos jogadores e os desafios enfrentados.
        * Os locais visitados, suas características e a atmosfera geral.
        * As pistas descobertas e sua relevância para a narrativa.
        * Como os eventos do arco atual levam logicamente ao próximo arco.
        * Use um tom envolvente e mantenha a coesão narrativa.

    7. Finalize com um cliffhanger ou um evento intrigante que gere suspense e conduza diretamente ao próximo arco, criando uma transição lógica e empolgante.

    ## Saida
    Você deve responder em formato JSON seguindo o seguinte esquema:\n"""

    schema_json_string = """ 
    {
        "title": "O título principal do arco atual, resumindo sua temática ou objetivo central, começando com: 'Arco: ...'.",
        "objective": "Objetivo principal do arco, descrevendo o foco e a meta dos jogadores.",
        "scenes": [
            {
            "local": {
                "name": "O nome do local onde a cena atual ocorre.",
                "description": "Uma descrição detalhada do local, incluindo suas características marcantes e atmosfera."
            },
            "events": [
                "Evento 1 Uma descrição dos eventos que ocorrem nesta cena, descrevendo desafios, encontros ou revelações importantes.",
                "Evento 2 Uma descrição dos eventos que ocorrem nesta cena, descrevendo desafios, encontros ou revelações importantes."
            ]
            },
            {
            "local": {
                "name": "O nome de outro local onde a cena atual ocorre.",
                "description": "Uma descrição detalhada do segundo local, incluindo suas características marcantes e atmosfera."
            },
            "events": [
                "Evento 1 Uma descrição dos eventos que ocorrem nesta cena, descrevendo desafios, encontros ou revelações importantes.",
                "Evento 2 Uma descrição dos eventos que ocorrem nesta cena, descrevendo desafios, encontros ou revelações importantes."
            ]
            }
        ],
        "summary": "Um detalhado resumo narrativo que conecta as cenas e explica o progresso geral do arco, em 6 frases.",
        "cliffhanger": "Gatilho narrativo que gera suspense e prepara para o próximo arco."
    }
    """

    aventura_schema = json.loads(schema_json_string)

    class Place(BaseModel):
        name: str
        description: str

    class Scenes(BaseModel):
        local: Place
        events: List[str]

    class Arco(BaseModel):
        title: str
        objective: str
        scenes: List[Scenes]    
        summary: str  
        cliffhanger: str



    system_prompt = """Você é um Dungeon Master especialista em criação de histórias para RPG. Responda no formato JSON seguindo o esquema infomrado abaixo."""
    tentativas=0
    arcos = model_response(system_prompt, prompt_arcos, aventura_schema, Arco)
    lista_aventura.append(arcos.model_dump())
    if (arco_atual+1) < 4:
        lista_arcos[arco_atual+1] = f"""## summary of the previous arc:\n{arcos.summary}\n\n{lista_arcos[arco_atual+1]}"""

Arco: 1
Arco: 2
Failed after 1 attempts. Last error: 400 Client Error: Bad Request for url: https://api.groq.com/openai/v1/chat/completions


AttributeError: 'NoneType' object has no attribute 'model_dump'

In [47]:
i = 0
lista_aventura[i]

{'title': 'Arco: Uma Jornada em Castrum Auremagnus',
 'objective': 'Os jogadores devem explorar a cidade de Castrum Auremagnus, coletar informações e tomar uma jornada perigosa e aventureira em busca da justiça e da luz.',
 'scenes': [{'local': {'name': 'Porta da Cidade',
    'description': 'A porta da cidade é uma estrutura de pedra alta e robusta que marca a entrada da cidade. Ela está rodeada por guardas bem armados e tem uma atmosfera de segurança e estabilidade.'},
   'events': ['O jogador é parado pelos guardas e deve explicar sua missão e intenções.',
    'O jogador é informado sobre a existência de uma vila chamada Urbs Luminaris, onde se encontram pistas importantes sobre a jornada.']},
  {'local': {'name': 'Café da Cidade',
    'description': 'O café é um local aconchegante e animado, onde as pessoas se reúnem para conversar e compartilhar histórias.'},
   'events': ['O jogador encontra um sussurro sobre uma escola de magia localizada fora da cidade que oferece informações pr

In [48]:
print(lista_arcos[i])

## Arco Atual
Apresentação do cenário iniciante na cidade de Castrum Auremagnus
Partir para uma jornada perigosa e aventureira em busca da justiça e da luz
O protagonista ganha a fama de seu heroísmo e conhecimento

## Próximo Arco
Encontrar pistas na vila de Urbs Luminaris e reconhecer que o Mundo segue por um caminho perigoso
Escolher entre prosseguir em sua jornada sozinho ou procurar pelas opiniões dos seus amigos
O herói se aproxima mais da verdade que lhe trará a sua missão
